# Personal Protection Equipment Mask Detection model
This notebook shows how you can use SageMaker Python API to do the following:
 * deploy our model to create an endpoint
 * run Real Time inference on endpoint using local image
 * visualize  and save the prediction on original image
 * run Batch Transform job to perfom the inference on your data stored in Amazon S3 bucket

### 1. Start the trial or buy our VitechLab PPE model on AWS Marketplace

You can find our model here:

### 2. Initializing a model using AWS SageMaker Python API

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

sess = sage.Session()
role = get_execution_role()

In [ ]:
from utils import get_model_package_arn

model_package_arn = get_model_package_arn(sess.boto_region_name)

In [ ]:
model = sage.ModelPackage(
    role=role,
    model_package_arn=model_package_arn)

### 3. Generating Prediction using `RealTimePredictor`

#### Creating a new endpoint

In [ ]:
endpoint_name='ppe-mask-detection-model-example-endpoint'

model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

#### Generating predictions

In [ ]:
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sess, 
    content_type="image/jpeg"
)

In [ ]:
file_name = 'sample_data/demo_input/image2.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    image_bytes = bytearray(f)

prediction_result = predictor.predict(image_bytes).decode("utf-8")

In [ ]:
import json

result = json.loads(prediction_result)
result

In [ ]:
from utils import visualize_detection

visualize_detection(file_name, result)

_Photo by Susan A. Romano_

### Delete the Endpoint
Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [ ]:
sess.delete_endpoint(predictor.endpoint)

### Generate Prediction using Batch Transform

#### Uploading input data to S3

In [ ]:
input_data_folder = 'sample_data/demo_input'
s3_data_folder = 'ppe_mask_detection_model_batch_testing/input'

input_data_location = sess.upload_data(input_data_folder, key_prefix=s3_data_folder)
output_data_location = input_data_location.replace('input', 'output')

#### Starting a new Batch Transform Job

In [ ]:
transformer = model.transformer(instance_count=1,
                               instance_type='ml.c4.2xlarge',
                               output_path=output_data_location,
                               strategy='SingleRecord',
                               assemble_with=None,
                               accept='image/jpeg')

In [ ]:
transformer.transform(input_data_location, 
    content_type='image/jpeg',
    compression_type=None,
    split_type=None,
    join_source=None)

transformer.wait()

#### Download and display the result file

In [ ]:
output_file_name = 'image1.jpg.out'
dest_path = 'sample_data/demo_raw_output/' + output_file_name
s3_path = 'ppe_mask_detection_model_batch_testing/output/' + output_file_name

bucket = sess.boto_session.resource('s3').Bucket(sess.default_bucket())
bucket.download_file(s3_path, dest_path)

with open(dest_path, 'r') as file:
    print(file.read())